In [24]:
import numpy as np

import aadc
from aadc.scipy.interpolate import interp1d, CubicSpline

N = 10

def func(t, xs, ys):
    yt = np.interp(t, xs, ys)
    t2 = 0.25
    yt2 = np.interp(t2, xs, ys)
    return yt+yt2

with aadc.record_kernel() as kernel:
    xs_numpy = np.linspace(0,1,N)
    ys_numpy = np.linspace(0,1,N)
    xs = aadc.array(xs_numpy)
    ys = aadc.array(ys_numpy)
    xs_args = xs.mark_as_input()
    ys_args = ys.mark_as_input()
    t = xs.mean()
    arg_t = t.mark_as_input()
    res = func(t, xs, ys)
    out_res = res.mark_as_output()

t_test = 0.5

f = aadc.VectorFunctionWithAD(kernel, args=[arg_t], res=[out_res], param_args=np.r_[xs_args, ys_args])
f.set_params(np.r_[xs_numpy, ys_numpy])
value, jacobian = f.evaluate(np.array([t_test]))

assert np.allclose(value.squeeze(), func(t_test, xs_numpy, ys_numpy))

eps = 1e-3
fd_derivative = (func(t_test + eps, xs_numpy, ys_numpy) - func(t_test - eps, xs_numpy, ys_numpy)) / (2 * eps)
assert np.allclose(fd_derivative, jacobian.squeeze())

You are using evaluation version of AADC. Expire date is 20250201


In [25]:
def func_scipy(t, xs, ys):
    linear = interp1d(xs, ys, kind='linear')
    cubic_legacy = interp1d(xs, ys, kind='cubic')
    cubic_new = CubicSpline(xs, ys, bc_type='clamped')
    return linear(t) + cubic_legacy(t) + cubic_new(t)

# Main kernel recording with context manager
with aadc.record_kernel() as kernel:
    xs_numpy = np.linspace(0, 1, N)
    ys_numpy = np.linspace(0, 1, N)
    xs = aadc.array(xs_numpy)
    ys = aadc.array(ys_numpy)
    xs_args = xs.mark_as_input()
    ys_args = ys.mark_as_input()
    t = xs.mean()
    arg_t = t.mark_as_input()
    res = func_scipy(t, xs, ys)
    out_res = res.mark_as_output()

# Test the function
t_test = 0.5
f = aadc.VectorFunctionWithAD(kernel, args=[arg_t], res=[out_res], param_args=np.r_[xs_args, ys_args])
f.set_params(np.r_[xs_numpy, ys_numpy])
value, jacobian = f.evaluate(np.array([t_test]))

# Verify results
assert np.allclose(value.squeeze(), func_scipy(t_test, xs_numpy, ys_numpy))

# Check derivatives
eps = 1e-3
fd_derivative = (func_scipy(t_test + eps, xs_numpy, ys_numpy) - func_scipy(t_test - eps, xs_numpy, ys_numpy)) / (2 * eps)
assert np.allclose(fd_derivative, jacobian.squeeze())

You are using evaluation version of AADC. Expire date is 20250201
